In [114]:
# ==============================================================================
#  1. IMPORTS AND CONFIGURATION
# ==============================================================================
import pandas as pd
import sqlite3
import json
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Configuration ---
# <<<<<<< SET THE AGENT YOU WANT TO ANALYZE HERE >>>>>>>>>
AGENT_TO_ANALYZE = "dueling_ddqn"

# Define the path to your database file
PROJECT_ROOT = Path().resolve()
DB_PATH = PROJECT_ROOT / "performance.db"

# --- Color Mapping for Metrics ---
# This dictionary allows you to define a consistent color for each potential metric.
METRIC_COLORS = {
    "Epsilon": "firebrick",
    "Policy Loss": "darkorange",
    "Avg Entropy": "green",
    "Actor Loss": "purple",
    "Critic Loss": "teal"
}


# ==============================================================================
#  2. DATA LOADING AND PREPARATION
# ==============================================================================

def load_and_prepare_data(db_path, agent_name):
    """
    Connects to the DB, loads data for the agent, and dynamically
    parses all metrics from the JSON column into new DataFrame columns.
    """
    if not db_path.exists():
        print(f"ERROR: Database file not found at '{db_path}'.")
        print("Please run a training session first using 'make train'.")
        return pd.DataFrame(), []
    
    try:
        conn = sqlite3.connect(db_path)
        query = f"SELECT epoch, average_score, metrics_json FROM training WHERE agent = '{agent_name}' ORDER BY epoch"
        df = pd.read_sql_query(query, conn)
        conn.close()
    except Exception as e:
        print(f"An error occurred while reading the database: {e}")
        return pd.DataFrame(), []

    if df.empty:
        print(f"No training records found for agent '{agent_name}'.")
        return pd.DataFrame(), []
    
    print(f"Successfully loaded {len(df)} records for agent '{agent_name}'.")

    # --- Dynamic Metric Extraction ---
    try:
        metrics_data = df['metrics_json'].apply(json.loads)
        metrics_df = pd.DataFrame(metrics_data.tolist())
        discovered_metrics = metrics_df.columns.tolist()
        print(f"Discovered metrics: {discovered_metrics}")
        df = pd.concat([df.drop(columns=['metrics_json']), metrics_df], axis=1)
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Could not parse metrics_json: {e}. Skipping metrics plot.")
        return df, []
        
    return df, discovered_metrics

# Load the data
df, metric_names = load_and_prepare_data(DB_PATH, AGENT_TO_ANALYZE)


# ==============================================================================
#  3. DYNAMIC VISUALIZATION
# ==============================================================================

if not df.empty:
    # Initialize a figure with a secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # --- Add the Average Score Trace (Primary Y-axis) ---
    fig.add_trace(
        go.Scatter(
            x=df['epoch'], 
            y=df['average_score'],
            name='Average Score',
            mode='lines',
            line=dict(color='royalblue', width=2, shape='spline'),
        ),
        secondary_y=False,
    )

    # --- Dynamically Add Traces for All Discovered Metrics ---
    if metric_names:
        for metric in metric_names:
            fig.add_trace(
                go.Scatter(
                    x=df['epoch'],
                    y=df[metric],
                    name=metric,
                    mode='lines',
                    line=dict(color=METRIC_COLORS.get(metric, 'grey'), width=2, shape='spline'),
                ),
                secondary_y=True,
            )

    # --- Customize the Layout for a Professional Look ---
    fig.update_layout(
        title_text=None,
        xaxis_title=None,
        template='plotly_white',
        
        # --- THIS IS THE FIX ---
        # This configuration aligns the legend to the top-left, outside the plot area.
        legend=dict(
            orientation="h",      # Horizontal layout
            yanchor="bottom",     # Anchor the legend to its bottom edge
            y=1.02,               # Position it just above the plot area
            xanchor="left",       # Anchor the legend to its left edge
            x=0                   # Position it at the far left of the plotting area
        ),
        
        yaxis=dict(range=[-50, 1050])
    )

    # Set y-axis titles
    fig.update_yaxes(title_text="<b>Average Score</b> (last 10 episodes)", secondary_y=False)
    fig.update_yaxes(title_text=None, secondary_y=True)

    # Show the interactive plot
    fig.show()
else:
    print("DataFrame is empty. Cannot generate plot.")

Successfully loaded 1000 records for agent 'dueling_ddqn'.
Discovered metrics: ['Epsilon']


---

In [128]:
# ==============================================================================
#  1. IMPORTS AND CONFIGURATION
# ==============================================================================
import pandas as pd
import sqlite3
import json
from pathlib import Path
import plotly.graph_objects as go
import numpy as np

# --- Configuration ---
# <<<<<<< SET THE AGENTS YOU WANT TO ANALYZE HERE >>>>>>>>>
AGENTS_TO_ANALYZE = [
    "qlearning", "dqn", "dqn_target", "ddqn", "dueling_ddqn", 
    "reinforce", "a2c", "ppo"
]

# --- Data Smoothing Parameter ---
# Groups N episodes together and averages their score to simplify the chart.
# A value of 1 means no smoothing.
EPISODE_MULTIPLIER = 10

# Define the path to your database file
PROJECT_ROOT = Path().resolve()
DB_PATH = PROJECT_ROOT / "performance.db"

# --- Pastel Color Mapping for All Agents ---
AGENT_COLORS = {
    "qlearning": "mediumseagreen",
    "dqn": "lightgreen",
    "dqn_target": "mediumturquoise",
    "ddqn": "cornflowerblue",
    "dueling_ddqn": "royalblue",
    "reinforce": "orange",
    "a2c": "mediumpurple",
    "ppo": "darkviolet"
}


# ==============================================================================
#  2. DATA LOADING AND PREPARATION WITH GUARANTEED FINAL POINT
# ==============================================================================

def load_and_smooth_data(db_path, agent_list, multiplier):
    """
    Connects to the DB, loads 'average_score' for all specified agents,
    and applies smoothing while guaranteeing the very last data point is preserved.
    """
    if not db_path.exists():
        print(f"ERROR: Database file not found at '{db_path}'.")
        print("Please run training for all agents first.")
        return {}
    
    agent_data = {}
    
    try:
        conn = sqlite3.connect(db_path)
        print("Loading and smoothing data from database...")
        for agent_name in agent_list:
            query = f"SELECT epoch, average_score FROM training WHERE agent = '{agent_name}' ORDER BY epoch"
            df = pd.read_sql_query(query, conn)
            
            if df.empty:
                print(f"- WARNING: No records found for '{agent_name}'")
                continue

            # --- NEW, CORRECTED SMOOTHING LOGIC ---
            if multiplier > 1 and len(df) > multiplier:
                # 1. Isolate the very last data point from the original data
                last_point = df.iloc[-1:].copy()
                
                # 2. Select all data EXCEPT the last point for smoothing
                df_to_smooth = df.iloc[:-1]
                
                # 3. Group and aggregate the main part of the data
                grouped = df_to_smooth.groupby(np.arange(len(df_to_smooth)) // multiplier)
                smoothed_part = grouped.agg(
                    epoch=('epoch', 'last'),
                    average_score=('average_score', 'mean')
                ).reset_index(drop=True)
                
                # 4. Concatenate the smoothed data with the original last point
                final_df = pd.concat([smoothed_part, last_point], ignore_index=True)
                
                agent_data[agent_name] = final_df
                print(f"- Loaded and processed {len(df)} records to {len(final_df)} points for '{agent_name}'")
            else:
                # If no smoothing is needed, just use the original DataFrame
                agent_data[agent_name] = df
                print(f"- Loaded {len(df)} records for '{agent_name}' (no smoothing)")
        
        conn.close()
    except Exception as e:
        print(f"An error occurred while reading the database: {e}")
        return {}
        
    return agent_data

# Load and process the data for all agents
all_data = load_and_smooth_data(DB_PATH, AGENTS_TO_ANALYZE, EPISODE_MULTIPLIER)


# ==============================================================================
#  3. COMPARATIVE VISUALIZATION
# ==============================================================================

if all_data:
    # Initialize a new figure
    fig = go.Figure()

    # --- Dynamically Add a Trace for Each Agent ---
    for agent_name, df in all_data.items():
        agent_color = AGENT_COLORS.get(agent_name, 'grey')
        
        # 1. Add the main spline line trace
        fig.add_trace(
            go.Scatter(
                x=df['epoch'], 
                y=df['average_score'],
                name=agent_name.upper(),
                mode='lines',
                line=dict(shape='spline', color=agent_color, width=2),
                opacity=0.9
            )
        )
        
        # 2. Add markers for "solved" points (score of 1000)
        solved_points = df[df['average_score'] >= 1000]
        if not solved_points.empty:
            fig.add_trace(
                go.Scatter(
                    x=solved_points['epoch'],
                    y=solved_points['average_score'],
                    mode='markers',
                    marker=dict(color=agent_color, size=7, symbol='circle'),
                    name=f'{agent_name.upper()} Solved',
                    showlegend=False # Hide this from the legend
                )
            )

    # --- Customize the Layout for a Professional Look ---
    fig.update_layout(
        title_text=f"<b>Comparative Training Performance (Smoothed by {EPISODE_MULTIPLIER} Episodes)</b>",
        title_x=0.5,
        xaxis_title=None, # Remove x-axis title
        yaxis_title=f"Smoothed Average Score",
        template='plotly_white',
        
        # --- Legend Position: Top-Left, Single Line ---
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0
        ),
        
        yaxis=dict(range=[-50, 1050])
    )

    # Show the interactive plot
    fig.show()
else:
    print("No data was loaded. Cannot generate plot.")

Loading and smoothing data from database...
- Loaded and processed 389 records to 40 points for 'qlearning'
- Loaded and processed 1000 records to 101 points for 'dqn'
- Loaded and processed 799 records to 81 points for 'dqn_target'
- Loaded and processed 979 records to 99 points for 'ddqn'
- Loaded and processed 1000 records to 101 points for 'dueling_ddqn'
- Loaded and processed 41 records to 5 points for 'reinforce'
- Loaded and processed 413 records to 43 points for 'a2c'
- Loaded and processed 68 records to 8 points for 'ppo'


---

In [130]:
# ==============================================================================
#  1. IMPORTS AND CONFIGURATION
# ==============================================================================
import pandas as pd
import sqlite3
from pathlib import Path
import plotly.graph_objects as go

# --- Configuration ---
# Define the path to your database file
PROJECT_ROOT = Path().resolve()
DB_PATH = PROJECT_ROOT / "performance.db"

# --- Pastel Color Mapping (Consistent with previous chart) ---
AGENT_COLORS = {
    "qlearning": "mediumseagreen",
    "dqn": "lightgreen",
    "dqn_target": "mediumturquoise",
    "ddqn": "cornflowerblue",
    "dueling_ddqn": "royalblue",
    "reinforce": "orange",
    "a2c": "mediumpurple",
    "ppo": "darkviolet"
}


# ==============================================================================
#  2. DATA LOADING AND PREPARATION
# ==============================================================================

def load_runtime_data(db_path):
    """
    Connects to the DB, loads the runtime data for all agents, calculates
    the duration in minutes, and prepares it for plotting.
    """
    if not db_path.exists():
        print(f"ERROR: Database file not found at '{db_path}'.")
        return pd.DataFrame()
    
    try:
        conn = sqlite3.connect(db_path)
        query = "SELECT agent, start_time, end_time, status FROM runtime WHERE end_time IS NOT NULL"
        df = pd.read_sql_query(query, conn)
        conn.close()
    except Exception as e:
        print(f"An error occurred while reading the database: {e}")
        return pd.DataFrame()

    if df.empty:
        print("No completed training runs found in the database.")
        return pd.DataFrame()
    
    print(f"Successfully loaded {len(df)} completed run records.")

    # --- Data Processing ---
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])
    
    # Calculate duration in seconds, then convert to minutes for plotting
    df['duration_seconds'] = (df['end_time'] - df['start_time']).dt.total_seconds()
    df['duration_minutes'] = df['duration_seconds'] / 60.0
    
    # We only want the *latest* run for each agent.
    df = df.sort_values('start_time').drop_duplicates(subset='agent', keep='last')
    
    # Sort by duration for the chart display (shortest on top)
    df_sorted = df.sort_values('duration_minutes', ascending=False)
        
    return df_sorted

# Load and process the runtime data
runtime_df = load_runtime_data(DB_PATH)

# Display the prepared data for verification
if not runtime_df.empty:
    display(runtime_df[['agent', 'duration_minutes', 'status']])


# ==============================================================================
#  3. VISUALIZATION
# ==============================================================================

if not runtime_df.empty:
    # Create a list of colors in the correct order for the sorted bars
    bar_colors = [AGENT_COLORS.get(agent, 'grey') for agent in runtime_df['agent']]

    # Initialize a new figure
    fig = go.Figure()

    # --- Add the Horizontal Bar Chart Trace (using minutes) ---
    fig.add_trace(
        go.Bar(
            x=runtime_df['duration_minutes'],
            y=runtime_df['agent'].str.upper(),
            orientation='h',
            marker=dict(color=bar_colors),
            text=runtime_df['duration_minutes'].apply(lambda x: f'{x:.1f} min'),
            textposition='outside',
        )
    )

    # --- Customize the Layout for a Professional Look ---
    fig.update_layout(
        title_text="<b>Total Training Duration by Agent</b>",
        title_x=0.5,
        xaxis_title=None, # Remove x-axis title
        yaxis_title=None,
        template='plotly_white',
        showlegend=False,
        
        # This sorts the chart from shortest duration (top) to longest (bottom)
        yaxis=dict(autorange="reversed"),
        
        # --- FIX: Increase left margin for more space ---
        margin=dict(l=150, r=40, t=80, b=50) 
    )

    # Show the plot
    fig.show()
else:
    print("DataFrame is empty. Cannot generate plot.")

Successfully loaded 16 completed run records.


,agent,duration_minutes,status
11,dueling_ddqn,158.150000,max_episodes_reached
8,dqn,146.916667,max_episodes_reached
10,ddqn,137.033333,solved
9,dqn_target,118.216667,solved
14,a2c,17.350000,solved
15,ppo,15.566667,solved
12,reinforce,9.166667,solved
7,qlearning,0.566667,solved


---

In [136]:
# ==============================================================================
#  1. IMPORTS AND CONFIGURATION
# ==============================================================================
import pandas as pd
import sqlite3
import json
from pathlib import Path
import plotly.graph_objects as go
import numpy as np

# --- Configuration ---
# All agents will be plotted.
AGENTS_TO_ANALYZE = [
    "qlearning", "dqn", "dqn_target", "ddqn", "dueling_ddqn", 
    "reinforce", "a2c", "ppo"
]

# Data smoothing can be adjusted for aesthetic purposes.
EPISODE_MULTIPLIER = 20

# Define the path to your database file
PROJECT_ROOT = Path().resolve()
DB_PATH = PROJECT_ROOT / "performance.db"

# --- Neon Color Mapping for Cyberpunk Style ---
AGENT_COLORS = {
    "qlearning": "#39ff14",     # Neon Green
    "dqn": "#a0ff99",           # Light Neon Green
    "dqn_target": "#00ffff",    # Cyan / Aqua
    "ddqn": "#4d4dff",          # Neon Blue
    "dueling_ddqn": "#8a2be2",   # Blue-Violet
    "reinforce": "#ff9900",     # Neon Orange
    "a2c": "#ff00ff",           # Magenta / Fuchsia
    "ppo": "#d533ff"            # Neon Purple
}

# --- Visual Styling Configuration ---
TARGET_SCORE_FOR_SUCCESS = 1000
SUCCESSFUL_LINE_WIDTH = 3.0
UNSUCCESSFUL_LINE_WIDTH = 1.5
SUCCESSFUL_OPACITY = 1.0
UNSUCCESSFUL_OPACITY = 0.65


# ==============================================================================
#  2. DATA LOADING AND PREPARATION
# ==============================================================================

def load_and_smooth_data(db_path, agent_list, multiplier):
    """
    Connects to the DB, loads 'average_score' for all specified agents,
    and applies smoothing while guaranteeing the very last data point is preserved.
    """
    if not db_path.exists():
        print(f"ERROR: Database file not found at '{db_path}'.")
        print("Please run training for all agents first.")
        return {}
    
    agent_data = {}
    
    try:
        conn = sqlite3.connect(db_path)
        print("Loading and smoothing data from database...")
        for agent_name in agent_list:
            query = f"SELECT epoch, average_score FROM training WHERE agent = '{agent_name}' ORDER BY epoch"
            df = pd.read_sql_query(query, conn)
            
            if df.empty:
                print(f"- WARNING: No records found for '{agent_name}'")
                continue

            if multiplier > 1 and len(df) > multiplier:
                last_point = df.iloc[-1:].copy()
                df_to_smooth = df.iloc[:-1]
                grouped = df_to_smooth.groupby(np.arange(len(df_to_smooth)) // multiplier)
                smoothed_part = grouped.agg(epoch=('epoch', 'last'), average_score=('average_score', 'mean')).reset_index(drop=True)
                final_df = pd.concat([smoothed_part, last_point], ignore_index=True)
                agent_data[agent_name] = final_df
            else:
                agent_data[agent_name] = df
        
        conn.close()
    except Exception as e:
        print(f"An error occurred while reading the database: {e}")
        return {}
        
    return agent_data

# Load and process the data for all agents
all_data = load_and_smooth_data(DB_PATH, AGENTS_TO_ANALYZE, EPISODE_MULTIPLIER)


# ==============================================================================
#  3. CYBERPUNK VISUALIZATION
# ==============================================================================

if all_data:
    # Initialize a new figure
    fig = go.Figure()

    # --- Add a Trace for Each Agent with a "Glow" Effect ---
    for agent_name, df in all_data.items():
        agent_color = AGENT_COLORS.get(agent_name, '#ffffff')
        
        # Determine styling based on whether the agent was successful
        if df['average_score'].max() >= TARGET_SCORE_FOR_SUCCESS:
            line_width = SUCCESSFUL_LINE_WIDTH
            opacity = SUCCESSFUL_OPACITY
        else:
            line_width = UNSUCCESSFUL_LINE_WIDTH
            opacity = UNSUCCESSFUL_OPACITY

        # Add a thicker, semi-transparent line underneath to simulate a glow
        fig.add_trace(
            go.Scatter(
                x=df['epoch'], 
                y=df['average_score'],
                mode='lines',
                line=dict(shape='spline', color=agent_color, width=line_width * 3),
                opacity=0.2 * opacity, # Glow opacity is a fraction of the main opacity
                showlegend=False
            )
        )
        
        # Add the main, sharp line on top
        fig.add_trace(
            go.Scatter(
                x=df['epoch'], 
                y=df['average_score'],
                mode='lines',
                line=dict(shape='spline', color=agent_color, width=line_width),
                opacity=opacity,
                showlegend=False
            )
        )

    # --- Strip All Layout Elements for a Purely Visual Cover Image ---
    fig.update_layout(
        # Set a dark background
        plot_bgcolor='black',
        paper_bgcolor='black',
        
        width=1200,
        height=600,
        
        # Remove all titles, legends, and axes
        title_text=None,
        showlegend=False,
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        
        # Remove all margins to make the chart fill the entire image space
        margin=dict(l=0, r=0, t=0, b=0)
    )

    # Show the final image
    fig.show()
else:
    print("No data was loaded. Cannot generate image.")

Loading and smoothing data from database...
